# Sentiment Analysis

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 29.9 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
pip install -U huggingface_hub

### Modules
* Hugging Face Datasets: This library provides methods for loading and preprocessing various NLP datasets. The load_dataset function allows you to load pre-existing datasets from the Hugging Face Hub or your local storage. The DatasetDict and Dataset classes represent different structures for organizing datasets.
* Transformers: This library provides pre-trained models and various functionalities for performing NLP tasks. Functions like AutoTokenizer are used to prepare text data for model input, while AutoModelForSequenceClassification defines a model structure for specific NLP tasks like sentiment analysis or text classification.
* PEFT: This library provides functionalities for efficient and scalable training of large language models. PefModel and PefConfig classes define the model architecture and hyperparameters for PEFT models. get_peft_model function helps instantiate a PEFT model, and LoraConfig defines specific configuration settings for the Lora architecture, which is a type of efficient parameterization used in large language models.
* Evaluate: This library provides tools for evaluating the performance of models on various NLP tasks, including metrics like accuracy, precision, recall, and F1-score.
* Torch: This framework is used for building and training neural networks. The code likely uses PyTorch for computational tasks like forward pass, backpropagation, and parameter updates.
* NumPy: This library provides efficient numerical operations used for various tasks in machine learning, including data manipulation and calculations.

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import logging

logger= logging.getLogger(__name__)
global_config = None

In [ ]:
# Base Model
model_checkpoint = "distilbert-base-uncased"

#define label maps
id2label = {0 : "Negative", 1 : "Positive"}
label2id = {"Negative" : 0, "Positive" : 1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels = 2,
                                                           id2label = id2label,
                                                           label2id = label2id,
                                                           )

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# checking if there are cpu or gpus
device_count = torch.cuda.device_count()
if device_count > 0:
  logger.debug("Select GPU device")
  device = torch.device("cuda")
else:
  logger.debug("Select CPU device")
  device = torch.device("cpu")

In [ ]:
device_count

1

In [ ]:
device

device(type='cuda')

In [ ]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
#loading the dataset
dataset = load_dataset("shawhin/imdb-truncated")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [ ]:
# Access the first example in the train split
example = dataset["train"]
text = example['text']
text[0]


'. . . or type on a computer keyboard, they\'d probably give this eponymous film a rating of "10." After all, no elephants are shown being killed during the movie; it is not even implied that any are hurt. To the contrary, the master of ELEPHANT WALK, John Wiley (Peter Finch), complains that he cannot shoot any of the pachyderms--no matter how menacing--without a permit from the government (and his tone suggests such permits are not within the realm of probability). Furthermore, the elements conspire--in the form of an unusual drought and a human cholera epidemic--to leave the Wiley plantation house vulnerable to total destruction by the Elephant People (as the natives dub them) to close the story. If you happen to see the current release EARTH, you\'ll detect the Elephant People are faring less well today.'

In [ ]:
example[36]

{'label': 0,
 'text': 'The danish movie "Slim Slam Slum" surprised me to be the worst movie i have seen to this date. I didn\'t think that it was possible to top my list of bad bad b-movies but this one deserves the gold. It\'s not funny. It\'s bad acting, It\'s bad filmed and the storyline is bad. The only positive thing i can say about this movie is it has three girls in it. I truly believe this flick has the potential to knock of the other danish movie "Stjerner uden hjerner" as the badest danish film ever made! And that\'s truly something. Congratulations in advance!'}

In [ ]:
validation = dataset['validation']
text_validation = validation['text']
text_validation[0]

'Disgused as an Asian Horror, "A Tale Of Two Sisters" is actually a complex character driven psychological drama, that engulfs the viewer into the problems of a seemingly normal family. I was really surprised at the depth of this movie. Director Ji-woon Kim\'s decision to focus more on telling a story rather than providing cheap scares, has proved a correct one. Creating one of the most ingenious new horror movies.<br /><br />"A Tale Of Two Sisters" tels the story, as it\'s name suggest of two sisters Su-mi and the younger Su-yeon, who after spending time in a mental institution return home to their father and apparently abusive stepmother. From then on we witness how the sisters deal with their stepmother\'s gradually rising aggression and erratic behavior. To say what would happen next would be to be spoil the entire experience. So I\'ll just leave it at that.<br /><br />The plot is very tightly written. With the characters nicely fleshed out. Ji-woon Kim\'s focus on a small cast off

In [ ]:
# preprocess the text
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space = True)


# create a tokenize function
def tokenize_function(examples):
  # extract text
  text = example['text']

  #tokenize and truncate the text
  tokenizer.truncation_side = "left"  #this means that we truncate the data from the left
  tokenized_input = tokenizer(
      text,
      return_tensors = "np",
      truncation = True,
      max_length = 512
  )    # return_tensors in this case return the tensor in the form of numpy array
  return tokenized_input


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
len(tokenizer)

30522

In [ ]:
# add pad token if none exists
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  model.resize_token_embeddings(len(tokenizer))  #update the model to handle the additonal pad token


  """
  In this code snippets, we add a pad token in case the model\
   is not defined with a pad\
  token. The pad token is generally ignore by the model during\
  its calculation. padding refers to the process of adding special\
  tokens to shorter sequences to make them all the same length. This\
  happens when the model is expecting a fix size of input sequence.
  """

In [ ]:
# Tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched = True)
tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

#### It will have extra columns 'input_ids' and 'attention_mask'

In [ ]:
tokenized_dataset['train']['attention_mask'][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
# Create a data collator
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

"""
This code snippets create a\
padding with the length of the \
longer sequence.
"""

'\nThis code snippets create apadding with the length of the longer sequence.\n'

In [ ]:
#Evaluation metrics - To evaluate the performance of the model

#import accuracy evalutation metric
accuracy = evaluate.load("accuracy")

In [ ]:
# Defining a finction to pass into the trainer later
def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis = 1)
  return {"accuracy": accuracy.compute(predictions = predictions, references = labels)}

In [ ]:
# Evaluating the untrained base model
text_list = ["It was good", "Worse thing I have ever seen",
             "Better than the first one", "This is not worth watching",
             "This one is a pass"]




In [ ]:
model.eval()  #this syntax make sure that things like dropout is false

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
"""import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.eval()  # Put model in evaluation mode"""

'import torch\n\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\nmodel.to(device)\nmodel.eval()  # Put model in evaluation mode'

In [ ]:
'''print("Untrained Model Predictions: ")
for text in text_list:
    # Tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)
    # Compute logits
    with torch.no_grad():
        logits = model(inputs).logits
    # Convert logits to label
    predictions = torch.argmax(logits, dim=-1).tolist()[0]

    print(text + " - " + id2label[predictions])'''

'print("Untrained Model Predictions: ")\nfor text in text_list:\n    # Tokenize text\n    inputs = tokenizer.encode(text, return_tensors="pt").to(device)\n    # Compute logits\n    with torch.no_grad():\n        logits = model(inputs).logits\n    # Convert logits to label\n    predictions = torch.argmax(logits, dim=-1).tolist()[0]\n\n    print(text + " - " + id2label[predictions])'

In [ ]:
print("Untrained Model Predictions: ")
for text in text_list:
  #tokenize text
  inputs = tokenizer.encode(text, return_tensors = "pt").to(device)  #this code is use so that the inputs is on the same device
  #compute logits
  logits = model(inputs).logits
  #convert logits to label
  predictions = torch.argmax(logits)

  print(text + " - " + id2label[predictions.tolist()])

Untrained Model Predictions: 
It was good - Positive
Worse thing I have ever seen - Positive
Better than the first one - Positive
This is not worth watching - Positive
This one is a pass - Positive


In [ ]:
#  Training the model
peft_config = LoraConfig(task_type = "SEQ_CLS", #sequence classifiactopm
                         r = 5, # Intrinsic rank of trainable weight matrix ## 4
                         lora_alpha = 32, # this is like a learning rate
                         lora_dropout = 0.01, # probability of dropout
                         target_modules = ['q_lin']) # we apply lora to querry layer

* task_type="SEQ_CLS": This parameter specifies the type of task the model will be used for. In this case, it indicates a sequence classification task.

* r=4: The r parameter represents the rank used in the low-rank approximation. A lower rank can significantly reduce computational complexity while attempting to preserve the model's performance.

* lora_alpha=32: lora_alpha signifies a hyperparameter in the LoRA algorithm, controlling the trade-off between performance and computational complexity.

* lora_dropout=0.01: Dropout is a regularization technique used in neural networks to prevent overfitting. This parameter specifies the dropout rate in the LoRA layer, where 0.01 means a 1% dropout rate.

* target_modules=['q_lin']: This parameter defines the target modules in the neural network architecture where the low-rank adaptation will be applied. In this case, it's specified for the 'q_lin' module.

In [ ]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=5, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 638,210 || all params: 67,593,220 || trainable%: 0.94419233171611


In [ ]:
# Hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 5

In [ ]:
# defining training arguments
training_args = TrainingArguments(
    output_dir = model_checkpoint + "-lora-text-classification",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
)

In [ ]:
# creater trainer object
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    tokenizer = tokenizer,
    data_collator = data_collator, #this will dynamicall pad examples in each batch to be equal length
    compute_metrics = compute_metrics
)

#train model
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.248615,{'accuracy': 0.903}
2,0.421700,0.125086,{'accuracy': 0.966}
3,0.421700,0.046878,{'accuracy': 0.99}
4,0.149800,0.029412,{'accuracy': 0.993}
5,0.149800,0.021378,{'accuracy': 0.996}


Trainer is attempting to log a value of "{'accuracy': 0.903}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.966}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.99}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.993}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.996}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This in

TrainOutput(global_step=1250, training_loss=0.24164473724365235, metrics={'train_runtime': 226.5616, 'train_samples_per_second': 22.069, 'train_steps_per_second': 5.517, 'total_flos': 556726126712160.0, 'train_loss': 0.24164473724365235, 'epoch': 5.0})

In [ ]:
print("Trained model predictions: ")
print("__________________")
model.to("cuda")
model.eval()
for text in text_list:
    # Tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)
    # Compute logits
    with torch.no_grad():
        logits = model(inputs).logits
    # Convert logits to label
    predictions = torch.argmax(logits, dim=-1).tolist()[0]

    print(text + " - " + id2label[predictions])

Trained model predictions: 
__________________
It was good - Positive
Worse thing I have ever seen - Negative
Better than the first one - Positive
This is not worth watching - Negative
This one is a pass - Negative


In [ ]:
# Define the directory where you want to save the model
final = "final"
directory_path = f"/content/distilbert-base-uncased_{final}"

# Save the model using the trainer's `save_model` method
trainer.save_model(directory_path)


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load the saved model
finetuned_model = AutoModelForSequenceClassification.from_pretrained(directory_path)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
finetuned_model.to(device)
finetuned_model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.01, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=5, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=5, out_features=768, bi

In [ ]:
t = "worse movie ever"
inputs = tokenizer.encode(t, return_tensors = 'pt').to(device)
lo = model(inputs).logits
pred = torch.argmax(lo, dim = -1).tolist()[0]
print(t + ": " + id2label[pred])

worse movie ever: Negative


In [ ]:
t = "who would have thought that this movie is actually not that good compared\
 to other movies but the movie is still up to par compared to others. "  # change between subpar and passable
inputs = tokenizer.encode(t, return_tensors = 'pt').to(device)
lo = finetuned_model(inputs).logits
pred = torch.argmax(lo, dim = -1).tolist()[0]
print(t + ": " + id2label[pred])

who would have thought that this movie is actually not that good compared to other movies but the movie is still up to par compared to others. : Positive


In [ ]:
model_name = "fine_tuned_distilbert"
finetuned_model.save_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


In [ ]:
mod.to(device)
mod.eval()
t = "worse movie ever"
inputs = tokenizer.encode(t, return_tensors = 'pt').to(device)
lo = finetuned_model(inputs).logits
pred = torch.argmax(lo, dim = -1).tolist()[0]
print(t + ": " + id2label[pred])

worse movie ever: Negative


In [ ]:
import shutil
import os

# Define the directory path where the model is saved
directory_path = "/content/fine_tuned_distilbert"

# Create a zip file of the saved model directory
shutil.make_archive(directory_path, 'zip', directory_path)

# Rename the zip file (optional)
os.rename(f"{directory_path}.zip", "/content/finetuned.zip")  # Rename if needed

# Download the zip file to your local machine
from google.colab import files
files.download("/content/finetuned.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>